In [1]:
from  numpy import array, histogram, polyfit, random
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook, output_file
output_notebook()
options = dict(plot_height=300, plot_width=600, tools="pan,wheel_zoom,reset,save,crosshair")

# this loads functions for interactions with the graphic
from ipywidgets import interact, fixed, FloatSlider, IntSlider, Button, Checkbox, HBox, Label

# this loads some H Vietoris topologyTML style files
from IPython.display import HTML, display, Latex
with open( './style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

Loading BokehJS ...

# Regressione lineare (semplice)

In un esperimento misuriamo coppie di valori $(x_i,y_i)$ per $i=1,\dots,n$.

Per esempio: (tempo, spazio), (temperatura, volume), ecc.

In teoria questi valori dovrebbero essere correlati linearmente: 

$$y_i=\beta_0+\beta_1 x_i$$

Però la misura delle $y$ è soggetta ad errori e la vera relazione tra le misure è

$$y_i=\beta_0+\beta_1 x_i+e_i$$

dove $e_i$ sono degli errori (ignoti).

Le $x_i$ si chiamano <mark>predittori</mark>.

Le $y_i$ si chiamano <mark>risposte</mark>.

Non esiste un metodo *ovvio* per inferire i parametri $\beta_0$ e $\beta_1$ dai dati $(x_i,y_i)$ per $i=1,\dots,n$. 

La scelta comune è di cercare quei parametri $\beta_0$ e $\beta_1$ che minimizzano le grandezze $e_i$. Però  generalmente non è possibile minimizzare *contemporaneamente* tutti gli errori. Dobbiamo quindi aggregare gli errori in un unico numero che rappresenti l'errore "totale".

La funzione anche aggrega gli errori si chiama <mark>cost function</mark>. Ci sono ininite possibililà. Per esempio, due scelte "ragionevoli" sono

1. minimizzare $$\sum_{i=1}^n |e_i|=\sum_{i=1}^n |y_i-\beta_0-\beta_1 x_i|$$

2. minimizzare $$\sum_{i=1}^n (e_i)^2=\sum_{i=1}^n (y_i-\beta_0-\beta_1 x_i)^2$$

Generalmente **si opta per seconda scelta** perché è un'espressione più semplice da trattare in maniera analitica. Quando i calcoli andavano fatti a mano questa era una scelta obbligata e l'abitudine si è consolidata negli anni. Inoltre, l'analisi statistica che faremo più sotto è notevolmente semplificata nel secondo caso.


Terminologia: spesso si scrive <mark>RSS</mark> (residual sum of squares) per il numero

$\displaystyle\sum_{{i=1}}^n e_i^2 = \sum_{i=1}^n (y_i-\beta_0-\beta_1 x_i)^2$

Esempio: consideriamo i dati in figura 

$$[x_1,\dots,x_9]=[0,\ 1,\ 2,\ 3,\ 4,\ 5,\ 6,\ 7,\ 8]$$

$$[y_1,\dots,y_9]=[1.5,\ 2,\ 3,\ 5,\ 4,\ 6.5,\ 8,\ 7,\ 9]$$

Un grafico come quello qui sotto si chiama <mark>scatterplot</mark>

In [2]:
x=array([0,1,2,3,4,5,6,7,8])
y=array([1.5,2,3,5,4,6.5,8,7,9])
fig=figure(**options,title="Un esempio di scatterplot",
                x_axis_label = "x",
                y_axis_label = "y",
                x_range=(-1, 10  ),      # range x da visualizzare inizialmente
                y_range=(-1, 10 ),       # range y da visualizzare inizialmente 
          )
fig.circle(x,y)
show(fig)

Una data una scelta di $\beta_0$ e $\beta_1$ gli errori $e_i$ sono quelli rappresentati dai segmenti blu in figura. Hanno segno positivo se il valore osservato staa sopra la retta interpolante, negativo se sgà sotto.

In [3]:
fig=figure(**options,
                x_range=(-1, 10  ),       # range x da visualizzare inizialmente
                y_range=(-1, 10 ),       # range y da visualizzare inizialmente 
          )
fig.circle(x,y) 
par = polyfit(x, y, 1, full=True)
slope=par[0][0]
intercept=par[0][1]
y_predicted = [slope*i + intercept  for i in x]
err1 = 0
err2 = 0
for (i,j,k) in zip(x,y,y_predicted):
    fig.line([i,i],[j,k], color='blue')
    err1 =err1 + abs(j-k)
    err2 =err2 + (j-k)**2
    
data = data = {'slope':slope, 'intercept':intercept, 'err1':err1, 'err2':err2}
    
fig.line(x,y_predicted, color='red', legend_label='y = {intercept:4.2f} + {slope:4.2f} x'.format(**data))
fig.legend.location = 'bottom_right'
display(Latex(
    '$\qquad\displaystyle\sum_{{i=1}}^n |e_i|=\ ${err1:4.2f}\
     $\qquad\displaystyle\sum_{{i=1}}^n e_i^2=\ ${err2:4.2f}\
     $\qquad\\beta_0=\ ${intercept:4.2f}\
     $\qquad\\beta_1=\ ${slope:4.2f}'.format(**data)))
show(fig)

<IPython.core.display.Latex object>

Per il grafico qui sopra ho scelto i valori di $\beta_0=1.34$ e $\beta_1=0.94$ che minimizzano la seconda cost function. Dal grafico qui sotto si può verificare numericamente che non minimizzano la orima cost function.

In [4]:
fig=figure(**options,
                x_range=(-1, 10  ),       # range x da visualizzare inizialmente
                y_range=(-1, 12 ),       # range y da visualizzare inizialmente 
          )
fig.circle(x,y)
par = polyfit(x, y, 1, full=True)
slope=par[0][0]
intercept=par[0][1]

line = fig.line(x,y_predicted, color='red', legend_label='y = β0 + β1 x')
fig.legend.location = 'bottom_right'
    
def update1(slope  = 0.8, intercept  =  1): 
    y_predicted = [slope*i + intercept  for i in x]
    line.data_source.data['x'] = x
    line.data_source.data['y'] = y_predicted
    err1 = 0
    err2 = 0
    for (i,j,k) in zip(x,y,y_predicted):
        err1 =err1 + abs(j-k)
        err2 =err2 + (j-k)**2
#     ab  = concatenate( ([a], linspace(a,b), [b]) )
#     pab = concatenate( ([0], tstudent.pdf(linspace(a,b),df), [0] ) )
#     p2.data_source.data['x'] =  ab
#     p2.data_source.data['y'] =  pab
#     pr =  tstudent.cdf(a,df) - tstudent.cdf(b,df)
#     err1 = 0
#     err2 = 0
#     data = {'err1':err1, 'err1':err2}
#     print('ε1 = {err1}    ε2 = {err2}\n'.format(**data) )
#     print('Pr(a<X<b) = tstudent.cdf({b}, {df}) - tstudent.cdf({a}, {df}) = {pr:04.2f}'.format(**data) )
    data = data = {'slope':slope, 'intercept':intercept, 'err1':err1, 'err2':err2}
    display(Latex(
    '$\qquad\displaystyle\sum_{{i=1}}^n |e_i|=\ ${err1:4.2f}                 \
     $\qquad\displaystyle\sum_{{i=1}}^n e_i^2=\ ${err2:4.2f}'.format(**data)))
    push_notebook()
    
show(fig, notebook_handle=True)
interact(update1,  
         slope     = FloatSlider(description="β1", min=0.8, max=1.2, step=0.05, value=0.8),
         intercept = FloatSlider(description="β0", min=1, max=1.6, step=0.05, value= 1),
         );

interactive(children=(FloatSlider(value=0.8, description='β1', max=1.2, min=0.8, step=0.05), FloatSlider(value…

<hr><hr>
Si osservi che è perfettamente ragionevole avere più valori di $y$ per la stessa $x$. Per esempio un altro set di misure potrebbe essere il seguente.

In [7]:
random.seed(2)
x  = random.choice(10, 1000)
err = random.randn(len(x))
y = 1.32 + 0.91 * x + err

fig=figure(**options,
                x_axis_label = "x",
                y_axis_label = "y",
                x_range=(-1, 10  ),       # range x da visualizzare inizialmente
                y_range=(-1, 12 ),       # range y da visualizzare inizialmente 
          )
fig.circle(x,y,radius=0.02) 
par = polyfit(x, y, 1, full=True)
slope=par[0][0]
intercept=par[0][1]
y_predicted = [slope*i + intercept  for i in x]
err2 = 0
residuals=list()
for (i,j,k) in zip(x,y,y_predicted):
    err2 =err2 + (j-k)**2
    residuals.append(j-k)
    
data = data = {'slope':slope, 'intercept':intercept, 'err2':err2}
    
fig.line(x,y_predicted, color='red', legend_label='y = {intercept:4.2f} + {slope:4.2f} x'.format(**data))
fig.legend.location = 'bottom_right'
display(Latex(
    '$\qquad\displaystyle\sum_{{i=1}}^n e_i^2=\ ${err2:4.2f}\
     $\qquad\\beta_0=\ ${intercept:4.2f}\
     $\qquad\\beta_1=\ ${slope:4.2f}'.format(**data)))
show(fig)

<IPython.core.display.Latex object>

Plottiamo un istogramma dei <mark>residui</mark> per vedere come sono distribuiti (per l'inferenza statistica qui sotto avremmo bisogno siamo verificate certe condizioni)

In [9]:
hist = figure(**options)
frequencies, edges = histogram(residuals, 30)
hist.quad(top=frequencies, bottom=0, left=edges[:-1], right=edges[1:], line_color="black")
show(hist)

<hr><hr>

L'espressione analitica per calcolare $\beta_0$ e $\beta_1$ ottenuti minimizzando $\displaystyle\sum_{{i=1}}^n e_i^2$ è la seguente (non serve memorizzare)

$$\beta_1=\displaystyle\frac{\sum_i^n (x_i-\bar x)(y_i-\bar y)}{\sum_i^n (x_i-\bar x)^2}$$  

$$\beta_0=\bar y - \beta_1\bar x$$

con l'usuale significato di $\bar y$ e $\bar x$.

## Inferenza statistica


Assumiamo che le v.a. $E_i$ che producono gli errori $e_i$ siano indipendenti, normali con media $\mu=0$ e deviazione standard $\sigma$ ignota.

Assumiamo che i predittori $x_i$ siano variabili deterministiche, cioè non soggette ad errore e conoscute con infinita precisone (ci si può sempre ridurre a questo caso).

Le risposte  $Y_i$ sono quindi delle v.a.

$$Y_i = \beta_0 + \beta_1 x_i + E_i$$

con distribuzione $N(\beta_0 + \beta_1 x_i,\sigma^2)$, dove $\beta_0$ e $\beta_1$ sono ignoti.

Gli stimatori dei parametri $\beta_0$ e $\beta_1$ ottenuti con il metodo dei minimi quadrati sono delle v.a. $B_0$, $B_1$ 

$B_1=\displaystyle\frac{\sum_i (x_i-\bar x)(Y_i-\bar Y)}{\sum_i (x_i-\bar x)^2}\qquad$  $B_0=\bar Y - B_1\bar x$


È possibile dimostrare che le seguenti v.a. *(non memorizzare)* hanno distribuzione $T(n-2)$

$\displaystyle\frac{B_0-\beta_0}{S_0}\qquad$ dove $\ \displaystyle S_0 = S\ \sqrt{\frac{\sum x_i^2}{\sum(x_i-\bar x)^2}}\quad$ ed $\ S = \displaystyle\sqrt{\frac1{n-2}\sum_{i=1}^n[y_i-(B_0+B_1x_i)]^2}$

$\displaystyle\frac{B_1-\beta_1}{S_1}\qquad$ dove $\ \displaystyle S_1 = S\ \sqrt{\frac1{\sum(x_i-\bar x)^2}}$



## Intervallo di confidenza per $\beta_0$ e $\beta_1$

Sia $t_{\alpha/2}$ tale che $\Pr(T<-t_{\alpha/2})=\alpha/2$ con $T\sim T(n-2)$

Quindi

$\displaystyle\Pr(-t_{\alpha/2}\le\frac{B_0-\beta_0}{S_0}\le t_{\alpha/2})=1-\alpha$

$\displaystyle\Pr(-t_{\alpha/2}\le\frac{B_1-\beta_1}{S_1}\le t_{\alpha/2})=1-\alpha$

Scrivo $s_0$ ed $s_1$ per il valore osservato di $S_0$ ed $S_1$.

Scrivo $b_0$ ed $b_1$ per il valore osservato di $B_0$ ed $B_1$ (le stime per $\beta_0$ e $beta_1$).

Abbiamo quindi che 

$\displaystyle\beta_0\in(b_0-s_0t_{\alpha/2},\ b_0+s_0t_{\alpha/2})$ con una confidenza $1-\alpha$

$\displaystyle\beta_1\in(b_1-s_1t_{\alpha/2},\ b_1+s_1t_{\alpha/2})$ con una confidenza $1-\alpha$

## Test di ipotesi su $\beta_1$

Dato un set di misure $(x_i,y_i)$, una domanda naturale è la seguente:

*Può essere che i valori della $x$ non abbiano una reale infuenza sui valori della $y$?*

In altre parole, è compatibile con i valori osservati che $\beta_1=0?$

Moltre librerie di statistica oltre a calcolare $\beta_0$ e $\beta_1$ e il loro intervallo di confidenza, calcolano il p-valore del segiente test di ipotesi 

$H_0:$ $\beta_1=0$ e $H_A=\beta_1\neq 0$

Si tratta di un t-test a due code chiamato <mark>marginal t-tests</mark>

Se il p-valore è grande ($>0.05$, ma dipende dai contesti) non si può rigettare con sufficiente confidenza l'ipotesi che l'apparente dipendenza della $y$ dalla $x$ sia dovuta al caso.

<hr><hr><hr><hr><hr>

# Regressione lineare multipla

Consideriamo ora il caso in cui abbiamo più di un predittore, per esempio abbiamo $p$ predittori.
Il modello è

$$y = \beta_0 + \beta_1x_1+\dots+\beta_px_p$$

In un esperimento misuriamo $p+1$ tuple di valori $(x_{1,i},\dots,x_{p,i}, y_i)$ per $i=1,\dots,n$.

Interpretiamo le risposte come i valori dati dal modello più errori casuali

$$y_i = \beta_0 + \beta_1 x_{1,i}+\dots+\beta_p x_{p,i} + e_i$$

I valori stimati di $\beta_0, \beta_1,\dots,\beta_p$ si ottengono (generalmente) minimizzando $\sum_i e_i^2$.

Il modello che si usa per l'inferenza statistica è simile a quello usato per la regressione semplice i predittori si interpretano come variabili deterministiche e le risposte sono v.a. funzione delle v.a. $E_i$

$$Y_i = \beta_0 + \beta_1 x_{1,i}+\dots+\beta_p x_{p,i} + E_i$$

Assumendo che le v.a. $E_i$ siano di tipo $N(0,\sigma^2)$ con $\sigma$ ignota si riesce a stimare un intervallo di confidenza per $\beta_0, \beta_1,\dots,\beta_p$.